#### Ingestion del archivo Leanguage_role

In [0]:
dbutils.widgets.text("p_enviroment","")
v_enviroment = dbutils.widgets.get("p_enviroment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/common_functions"

In [0]:
%run "../includes/configuration"

##### Paso 1 - Leer el archivo Json usando "DataFrameReader" en Spark

In [0]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType

In [0]:
leanguage_role_schema = StructType(fields = [
  StructField("roleId", IntegerType(), False),
  StructField("languageRole", StringType(),False)
])

In [0]:
language_df = spark.read.option("multiline","True").schema(leanguage_role_schema) \
                            .json(f"{bronze_folder_path}/{v_file_date}/language_role.json*")

#### Paso 2 - Agregar los campos faltnates
##### 1. Agregar "ingestion_date"
##### 2. Agregar "enviroment"

In [0]:
from pyspark.sql.functions import lit, current_timestamp

In [0]:
language_role_final = add_ingestion_date(language_df).withColumnRenamed("roleId","role_id") \
                                .withColumnRenamed("languageRole","language_role") \
                                .withColumn("enviromen",lit(v_enviroment)) \
                                .withColumn("file_date",lit(v_file_date))

#### Paso 4 - Agregar el DF en formato parQuet

In [0]:
language_role_final.write.mode("overwrite").format("delta").saveAsTable("movie_silver.language_role")

In [0]:
display(spark.read.format("delta").load(f"{silver_foler_path}/language_role"))

In [0]:
dbutils.notebook.exit("Success!")